## 데이터 그루핑

In [1]:
import pandas as pd
df = pd.read_csv('credit.csv')
df

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331
...,...,...,...,...,...,...,...,...,...,...,...
395,12.096,4100,307,3,32,13,Male,No,Yes,Caucasian,560
396,13.364,3838,296,5,65,17,Male,No,No,African American,480
397,57.872,4171,321,5,67,12,Female,No,Yes,Caucasian,138
398,37.728,2525,192,1,44,13,Male,No,Yes,Caucasian,0


## 그루핑의 전형적인 패턴

1. 범주형 변수 선정
2. 변수의 특정 값에 따라 데이터를 그루핑
3. 각 그룹에 대하여 연산 수행 (취합, 변환, 필터링)

## groupby 메서드

In [3]:
# 그루핑 기준이 되는 칼럼을 괄호 안에 넣는다
group = df.groupby('Gender')

## 1. 그룹별 취합연산

In [4]:
# 그룹별 사이즈
group.size()

Gender
Female    207
Male      193
dtype: int64

In [5]:
# 그룹별 통계량
group[['Income','Education']].mean()

,Income,Education
Gender,,
Female,44.853928,13.434783
Male,45.610316,13.466321


## 그룹별 특정 칼럼에 대한 작업

In [7]:
group['Limit'].mean()

Gender
Female    4756.516908
Male      4713.165803
Name: Limit, dtype: float64

In [8]:
group['Limit'].max()

Gender
Female    13414
Male      13913
Name: Limit, dtype: int64

## agg를 이용한 custom 취합연산

In [9]:
df.groupby('Ethnicity').agg({'Income':'mean'})

,Income
Ethnicity,
African American,47.682101
Asian,44.187833
Caucasian,44.521945


In [10]:
df.groupby('Ethnicity').agg({'Income':['mean','median']})

Income        
                       mean  median
Ethnicity                          
African American  47.682101  34.509
Asian             44.187833  30.059
Caucasian         44.521945  33.437

In [11]:
df.groupby('Ethnicity').agg({'Income':['mean','median'],'Limit':'max'})

Income          Limit
                       mean  median    max
Ethnicity                                 
African American  47.682101  34.509  13414
Asian             44.187833  30.059  12066
Caucasian         44.521945  33.437  13913

## agg와 사용자 정의함수 함께 사용

* agg에서 사용하는 함수의 주의점
    - 입력값: 데이터프레임의 칼럼
    - 출력값: 하나의 취합된 값

In [12]:
# 힌트: sum을 쓰면 조건을 만족하는 데이터의 갯수가 계산됨
(df['Cards']>3).sum()

123

In [13]:
# 카드가 3개 이상인 사람의 갯수를 리턴하는 함수
def cardnum(df_col): #x는 데이터프레임의 칼럼이라고 생각
    return (df_col>3).sum()

In [14]:
cardnum(df['Cards'])

123

In [15]:
df.groupby('Ethnicity').agg({'Cards':[cardnum,'size']})

Cards     
                 cardnum size
Ethnicity                    
African American      25   99
Asian                 32  102
Caucasian             66  199

## 2. 그룹별 변환 (transform, 사용자 정의함수)

* 그룹별로 변환을 다르게 하고싶을 때 사용
* transform에서 사용하는 함수의 주의점
    - 입력값: 데이터프레임의 칼럼(시리즈)
    - 출력값: 데이터프레임의 칼럼(시리즈)

In [16]:
import numpy as np
np.mean(df['Rating']) # 평균 신용점수

354.94

In [17]:
df.groupby('Ethnicity')['Rating'].transform(np.mean)

0      354.773869
1      345.431373
2      345.431373
3      345.431373
4      354.773869
          ...    
395    354.773869
396    365.070707
397    354.773869
398    354.773869
399    345.431373
Name: Rating, Length: 400, dtype: float64

In [19]:
df['avg_rating'] = df.groupby('Ethnicity')['Rating'].transform(np.mean)
df

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance,avg_rating
0,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333,354.773869
1,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903,345.431373
2,104.593,7075,514,4,71,11,Male,No,No,Asian,580,345.431373
3,148.924,9504,681,3,36,11,Female,No,No,Asian,964,345.431373
4,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331,354.773869
...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.096,4100,307,3,32,13,Male,No,Yes,Caucasian,560,354.773869
396,13.364,3838,296,5,65,17,Male,No,No,African American,480,365.070707
397,57.872,4171,321,5,67,12,Female,No,Yes,Caucasian,138,354.773869
398,37.728,2525,192,1,44,13,Male,No,Yes,Caucasian,0,354.773869


In [15]:
# 그룹별 정규분포 변환을 다르게 하고싶을때
# 일단 사용자 함수 정의
def normalize(df_col):
    return (df_col - df_col.mean())/df_col.std()


In [16]:
# 테스트
normalize(df['Income'])

0     -0.860505
1      1.725276
2      1.684646
3      2.942467
4      0.302549
         ...   
395   -0.939809
396   -0.903832
397    0.359012
398   -0.212542
399   -0.752403
Name: Income, Length: 400, dtype: float64

In [17]:
# 그룹별로 다르게 표준화 된 값이 나올 것임
df.groupby('Ethnicity')['Income'].transform(normalize)

0     -0.890389
1      1.736963
2      1.696739
3      2.941966
4      0.341362
         ...   
395   -0.974376
396   -0.884502
397    0.401160
398   -0.204153
399   -0.715908
Name: Income, Length: 400, dtype: float64

## 3. 그룹별 필터 (filter와 사용자 정의함수)

* 그룹별로 조건을 만족하는 데이터만 추리는 방법
* 사용자 함수 지정시 유의점
    - 입력값: 데이터프레임 전체라고 생각
    - 출력값: 불리언(참 또는 거짓)


In [ ]:
def filter_func(x):
    return x['Balance'].mean()>515

In [ ]:
filter_func(df)

True

In [ ]:
df.groupby('Ethnicity').filter(filter_func)

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
4,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331
5,80.180,8047,569,4,77,10,Male,No,No,Caucasian,1151
6,20.996,3388,259,2,37,12,Female,No,No,African American,203
8,15.125,3300,266,5,66,13,Female,No,No,Caucasian,279
...,...,...,...,...,...,...,...,...,...,...,...
394,49.794,5758,410,4,40,8,Male,No,No,Caucasian,734
395,12.096,4100,307,3,32,13,Male,No,Yes,Caucasian,560
396,13.364,3838,296,5,65,17,Male,No,No,African American,480
397,57.872,4171,321,5,67,12,Female,No,Yes,Caucasian,138


## 연습문제

* 타이타닉 데이터를 사용하여 다음 물음에 답하시오
    - 좌석 등급별 평균 가격을 구해보시오
    - 성인 남성인 사람과 그렇지 않은 사람의 생존률을 계산해보시오
    - 혼자 여행온 사람과 그렇지 않은 사람의 생존률에 차이가 있는지 확인해보시오
    - 나이를 어린이(<18) 성인(>18 그리고 <60) 노인(>60)의 세 그룹으로 나누고 그룹간에 생존률의 차이를 구해보시오